# Epidemic Simulation in a Central Force Field

In an epidemic simulation, a group of agents are allowed to randomly move. One of these agents will be infected and he can transmit that to neighbours within some radius around them. An infection is passed only to a *susceptible* agent. After a time, the *infected* agent become *recovered* and removed from the susceptible community.

Here, the agents are living in a central force field, $f(r)$. i.e., these agents are pulled towards a central location. In real life, this may correspond to a place where people visit regularly, such as a mall. In the force field, movement of the agents are given by the equations of motion, 

$$ \frac{{\rm d}{\bf x}_i}{{\rm d}t} = {\bf v}_i \quad \text{and} \quad \frac{{\rm d}{\bf v}_i}{{\rm d}t} = f({\bf x}_i) \hat{\bf x}_i $$

I will be using a *gaussian* force field of the form

$$ f(r) = -f_0 \exp \left( -\frac{(r / r_0)^2}{w} \right) $$

where $r = \vert {\bf x}_i - {\bf x}_f \vert$ with ${\bf x}_f$ is the centre of the force field.

In [38]:
import numpy as np

class community:
    """ a group of random walking agents """

    def __init__(self, n: int, v0: float = 1.0, lim: float = 100.0, rinf: float = 0.05, pinf: float = 0.1, fcentre: list = ..., f0: float = 10.0, w: float = 0.05, dt: float = 0.01) -> None:
        if fcentre is ... :
            fcentre = [lim / 2.0, lim / 2.0]
            
        self.n       = n                   # number of agents
        self.lim     = lim                 # space limits
        self.dt      = dt                  # time step for integration

        self.rinf    = self.lim * rinf     # infection radius
        self.pinf    = pinf                # infection probability
        
        self.fcentre    = np.asarray(fcentre) # centre of the force field
        self.f0, self.w = f0, w               # force parameters

        # create random positions
        self.pos = np.random.uniform(0.0, self.lim, (self.n, 2))

        # create random velocities
        angle    = np.random.uniform(0.0, 2*np.pi, self.n)
        self.vel = v0 * np.array([np.cos(angle), np.sin(angle)]).T

        # infection status: all are susceptible now
        self.state = np.zeros(self.n) # 0 -> susceptible, 1 -> infected, 2 -> recovered

        self.soft = 0.1 # softening value

    def move(self) -> None:
        """ move the agents by one step. """
        # using leapfrog integration
        self.pos = self.pos + self.vel * self.dt * 0.5 # postion half-step

        # get the acceleration / force
        if self.f0:
            sep = self.pos - self.fcentre
            r   = np.sqrt(np.sum(sep**2, -1) + self.soft**2)
            acc = self.get_force(r)[:, None] * sep # acceleration = force / mass

            self.vel = self.vel + acc * self.dt # velocity step

        self.pos = self.pos + self.vel * self.dt * 0.5 # postion half-step

        # put reflection boundary conditions
        mask            = (self.pos > self.lim) | (self.pos < 0.0) # crossing the boundary
        self.vel[mask] *= -1

    def get_force(self, r: float) -> float:
        """ calculate the force. """
        return -self.f0 * np.exp(-(r / self.lim)**2 / self.w) / r


In [47]:
import matplotlib.pyplot as plt
import matplotlib.animation as anim
plt.style.use('ggplot')

com = community(n = 100, dt = 0.05)

fig, ax = plt.subplots(figsize = [8,8])
ax.axis([0., com.lim, 0, com.lim])

line, = ax.plot([], [], 'o', ms = 3, color = 'black')

def animate(i):
    line.set_data(com.pos[:,0], com.pos[:,1])
    com.move()

ani = anim.FuncAnimation(fig, animate, frames = 500)
plt.close()


from IPython.display import HTML
HTML(ani.to_jshtml())


(-99.99800001999986, -6.42038481059891e-06)